In [1]:
import numpy as np
import scipy
import numba
import matplotlib
import seaborn

print("NumPy version:", np.__version__)
print("SciPy version:", scipy.__version__)
print("Numba version:", numba.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", seaborn.__version__)


NumPy version: 1.24.4
SciPy version: 1.13.1
Numba version: 0.60.0
Matplotlib version: 3.9.2
Seaborn version: 0.13.2


In [2]:
import pandas as pd
import numpy as np
import wrangle as w

import seaborn as sns
import matplotlib.pyplot as plt
import time

In [3]:
df = w.clean_data('nfl_data.csv')
df

,season,team,week,day,date,result,ot,host,opp,pts,...,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,0,3,3,6,155,7,14,0,0,31:30
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,1,4,4,2,23,5,12,3,4,28:40
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,5,2,2,2,164,5,12,0,0,36:28
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,2,3,3,6,137,7,16,0,0,32:10
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,1,3,3,7,206,5,14,0,1,28:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,2023,WAS,13,Sun,December 3,L,no,home,Miami Dolphins,15,...,1,6,6,3,127,7,13,2,2,32:16
12346,2023,WAS,15,Sun,December 17,L,no,away,Los Angeles Rams,20,...,3,2,2,3,134,9,16,0,0,35:53
12347,2023,WAS,16,Sun,December 24,L,no,away,New York Jets,28,...,3,3,3,6,292,4,16,1,2,36:16
12348,2023,WAS,17,Sun,December 31,L,no,home,San Francisco 49ers,10,...,2,3,3,2,76,4,9,0,1,38:13


In [4]:
df.shape

(12350, 63)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12350 entries, 0 to 12349
Data columns (total 63 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                12350 non-null  int64  
 1   team                  12350 non-null  object 
 2   week                  12350 non-null  int64  
 3   day                   12350 non-null  object 
 4   date                  12350 non-null  object 
 5   result                12350 non-null  object 
 6   ot                    12350 non-null  object 
 7   host                  12350 non-null  object 
 8   opp                   12350 non-null  object 
 9   pts                   12350 non-null  int64  
 10  opp_pts               12350 non-null  int64  
 11  pass_cmp              12350 non-null  int64  
 12  pass_att              12350 non-null  int64  
 13  pass_yds              12350 non-null  int64  
 14  pass_td               12350 non-null  int64  
 15  int                

In [6]:
df.iloc[:16]

,season,team,week,day,date,result,ot,host,opp,pts,...,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,0,3,3,6,155,7,14,0,0,31:30
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,1,4,4,2,23,5,12,3,4,28:40
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,5,2,2,2,164,5,12,0,0,36:28
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,2,3,3,6,137,7,16,0,0,32:10
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,1,3,3,7,206,5,14,0,1,28:57
5,2000,CRD,7,Sun,October 15,L,no,home,Philadelphia Eagles,14,...,2,3,4,3,42,10,18,2,2,39:34
6,2000,CRD,8,Sun,October 22,L,no,away,Dallas Cowboys,7,...,2,6,6,3,38,3,7,0,0,30:45
7,2000,CRD,9,Sun,October 29,L,no,home,New Orleans Saints,10,...,1,3,3,5,118,6,14,0,0,26:41
8,2000,CRD,10,Sun,November 5,W,no,home,Washington Redskins,16,...,5,0,0,0,0,8,15,0,1,37:26
9,2000,CRD,11,Sun,November 12,L,no,away,Minnesota Vikings,14,...,1,4,4,1,39,6,9,0,1,34:08


In [7]:
df[(df['season'] == 2000) & (df['team'].str.contains('nyg', case=False))]

,season,team,week,day,date,result,ot,host,opp,pts,...,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp
352,2000,NYG,1,Sun,September 3,W,no,home,Arizona Cardinals,21,...,1,1,1,5,105,6,15,0,2,28:30
353,2000,NYG,2,Sun,September 10,W,no,away,Philadelphia Eagles,33,...,2,0,1,4,82,3,10,0,1,21:16
354,2000,NYG,3,Sun,September 17,W,no,away,Chicago Bears,14,...,1,1,1,7,211,3,12,0,1,21:49
355,2000,NYG,4,Sun,September 24,L,no,home,Washington Redskins,6,...,2,1,2,5,88,5,14,0,0,31:22
356,2000,NYG,5,Sun,October 1,L,no,away,Tennessee Titans,14,...,1,4,4,4,66,14,20,0,1,42:46
357,2000,NYG,6,Sun,October 8,W,no,away,Atlanta Falcons,13,...,3,0,0,5,92,5,14,0,1,30:15
358,2000,NYG,7,Sun,October 15,W,no,home,Dallas Cowboys,19,...,0,2,2,3,39,5,13,0,1,30:42
359,2000,NYG,9,Sun,October 29,W,no,home,Philadelphia Eagles,24,...,1,1,1,9,300,1,11,0,1,16:19
360,2000,NYG,10,Sun,November 5,W,no,away,Cleveland Browns,24,...,1,0,0,6,176,5,13,0,1,23:20
361,2000,NYG,11,Sun,November 12,L,no,home,St. Louis Rams,24,...,2,5,5,6,144,3,13,1,1,36:40


In [38]:
df[df['team'].str.contains('NYG')]

,season,team,week,day,date,result,ot,host,opp,pts,...,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id
352,2000,NYG,1,Sun,September 3,W,no,home,Arizona Cardinals,21,...,1,1,5,105,6,15,0,2,28:30,2000_1_Arizona Cardinals_NYG
353,2000,NYG,2,Sun,September 10,W,no,away,Philadelphia Eagles,33,...,0,1,4,82,3,10,0,1,21:16,2000_2_NYG_Philadelphia Eagles
354,2000,NYG,3,Sun,September 17,W,no,away,Chicago Bears,14,...,1,1,7,211,3,12,0,1,21:49,2000_3_Chicago Bears_NYG
355,2000,NYG,4,Sun,September 24,L,no,home,Washington Redskins,6,...,1,2,5,88,5,14,0,0,31:22,2000_4_NYG_Washington Redskins
356,2000,NYG,5,Sun,October 1,L,no,away,Tennessee Titans,14,...,4,4,4,66,14,20,0,1,42:46,2000_5_NYG_Tennessee Titans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12209,2023,NYG,14,Mon,December 11,W,no,home,Green Bay Packers,24,...,1,1,3,119,5,14,1,1,31:06,2023_14_Green Bay Packers_NYG
12210,2023,NYG,15,Sun,December 17,L,no,away,New Orleans Saints,6,...,3,3,4,167,6,12,0,0,33:27,2023_15_NYG_New Orleans Saints
12211,2023,NYG,16,Mon,December 25,L,no,away,Philadelphia Eagles,25,...,3,3,2,108,8,15,2,2,34:38,2023_16_NYG_Philadelphia Eagles
12212,2023,NYG,17,Sun,December 31,L,no,home,Los Angeles Rams,25,...,2,4,5,287,2,8,0,1,30:11,2023_17_Los Angeles Rams_NYG


In [40]:
df['team'].unique().tolist()

['CRD',
 'ATL',
 'RAV',
 'BUF',
 'CAR',
 'CHI',
 'CIN',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GNB',
 'CLT',
 'JAX',
 'KAN',
 'SDG',
 'RAM',
 'RAI',
 'MIA',
 'MIN',
 'NWE',
 'NOR',
 'NYG',
 'NYJ',
 'PHI',
 'PIT',
 'SEA',
 'SFO',
 'TAM',
 'OTI',
 'WAS',
 'HTX']

In [41]:
# The mapping of team abbreviations to full names
team_code_map = {
    'CRD': 'Arizona Cardinals',
    'ATL': 'Atlanta Falcons',
    'RAV': 'Baltimore Ravens',
    'BUF': 'Buffalo Bills',
    'CAR': 'Carolina Panthers',
    'CHI': 'Chicago Bears',
    'CIN': 'Cincinnati Bengals',
    'CLE': 'Cleveland Browns',
    'DAL': 'Dallas Cowboys',
    'DEN': 'Denver Broncos',
    'DET': 'Detroit Lions',
    'GNB': 'Green Bay Packers',
    'CLT': 'Indianapolis Colts',
    'JAX': 'Jacksonville Jaguars',
    'KAN': 'Kansas City Chiefs',
    'SDG': 'Los Angeles Chargers',
    'RAM': 'Los Angeles Rams',
    'RAI': 'Las Vegas Raiders',
    'MIA': 'Miami Dolphins',
    'MIN': 'Minnesota Vikings',
    'NWE': 'New England Patriots',
    'NOR': 'New Orleans Saints',
    'NYG': 'New York Giants',
    'NYJ': 'New York Jets',
    'PHI': 'Philadelphia Eagles',
    'PIT': 'Pittsburgh Steelers',
    'SEA': 'Seattle Seahawks',
    'SFO': 'San Francisco 49ers',
    'TAM': 'Tampa Bay Buccaneers',
    'OTI': 'Tennessee Titans',
    'WAS': 'Washington Commanders',
    'HTX': 'Houston Texans'
}

# Step 1: Replace team abbreviations in the 'team' column with full names
df['team_full_name'] = df['team'].map(team_code_map)

In [91]:
df['name_x_season']= df['opp']

In [94]:
df['opp'].sort_values().unique().tolist()

['Arizona Cardinals',
 'Atlanta Falcons',
 'Baltimore Ravens',
 'Buffalo Bills',
 'Carolina Panthers',
 'Chicago Bears',
 'Cincinnati Bengals',
 'Cleveland Browns',
 'Dallas Cowboys',
 'Denver Broncos',
 'Detroit Lions',
 'Green Bay Packers',
 'Houston Texans',
 'Indianapolis Colts',
 'Jacksonville Jaguars',
 'Kansas City Chiefs',
 'Las Vegas Raiders',
 'Los Angeles Chargers',
 'Los Angeles Rams',
 'Miami Dolphins',
 'Minnesota Vikings',
 'New England Patriots',
 'New Orleans Saints',
 'New York Giants',
 'New York Jets',
 'Oakland Raiders',
 'Philadelphia Eagles',
 'Pittsburgh Steelers',
 'San Diego Chargers',
 'San Francisco 49ers',
 'Seattle Seahawks',
 'St. Louis Rams',
 'Tampa Bay Buccaneers',
 'Tennessee Titans',
 'Washington Commanders',
 'Washington Football Team',
 'Washington Redskins']

In [96]:
# Original teams
teams_original = [
    'Arizona Cardinals',
    'Atlanta Falcons',
    'Baltimore Ravens',
    'Buffalo Bills',
    'Carolina Panthers',
    'Chicago Bears',
    'Cincinnati Bengals',
    'Cleveland Browns',
    'Dallas Cowboys',
    'Denver Broncos',
    'Detroit Lions',
    'Green Bay Packers',
    'Houston Texans',
    'Indianapolis Colts',
    'Jacksonville Jaguars',
    'Kansas City Chiefs',
    'Las Vegas Raiders',
    'Los Angeles Chargers',
    'Los Angeles Rams',
    'Miami Dolphins',
    'Minnesota Vikings',
    'New England Patriots',
    'New Orleans Saints',
    'New York Giants',
    'New York Jets',
    'Oakland Raiders',  # Will be replaced
    'Philadelphia Eagles',
    'Pittsburgh Steelers',
    'San Diego Chargers',  # Will be replaced
    'San Francisco 49ers',
    'Seattle Seahawks',
    'St. Louis Rams',  # Will be replaced
    'Tampa Bay Buccaneers',
    'Tennessee Titans',
    'Washington Commanders',
    'Washington Football Team',  # Will be replaced
    'Washington Redskins'  # Will be replaced
]

# Mapping
team_mapping = {
    'Oakland Raiders': 'Las Vegas Raiders',
    'San Diego Chargers': 'Los Angeles Chargers',
    'St. Louis Rams': 'Los Angeles Rams',
    'Washington Football Team': 'Washington Commanders',
    'Washington Redskins': 'Washington Commanders'
}

# Update the original list to reflect current teams
updated_teams = [team_mapping.get(team, team) for team in teams_original]

# Remove duplicates
updated_teams = list(dict.fromkeys(updated_teams))

In [98]:
team_mapping = {
    'Arizona Cardinals': 'Arizona Cardinals',
    'Atlanta Falcons': 'Atlanta Falcons',
    'Baltimore Ravens': 'Baltimore Ravens',
    'Buffalo Bills': 'Buffalo Bills',
    'Carolina Panthers': 'Carolina Panthers',
    'Chicago Bears': 'Chicago Bears',
    'Cincinnati Bengals': 'Cincinnati Bengals',
    'Cleveland Browns': 'Cleveland Browns',
    'Dallas Cowboys': 'Dallas Cowboys',
    'Denver Broncos': 'Denver Broncos',
    'Detroit Lions': 'Detroit Lions',
    'Green Bay Packers': 'Green Bay Packers',
    'Houston Texans': 'Houston Texans',
    'Indianapolis Colts': 'Indianapolis Colts',
    'Jacksonville Jaguars': 'Jacksonville Jaguars',
    'Kansas City Chiefs': 'Kansas City Chiefs',
    'Las Vegas Raiders': 'Las Vegas Raiders',  # Updated
    'Los Angeles Chargers': 'Los Angeles Chargers',  # Updated
    'Los Angeles Rams': 'Los Angeles Rams',  # Updated
    'Miami Dolphins': 'Miami Dolphins',
    'Minnesota Vikings': 'Minnesota Vikings',
    'New England Patriots': 'New England Patriots',
    'New Orleans Saints': 'New Orleans Saints',
    'New York Giants': 'New York Giants',
    'New York Jets': 'New York Jets',
    'Oakland Raiders': 'Las Vegas Raiders',  # Will be replaced
    'Philadelphia Eagles': 'Philadelphia Eagles',
    'Pittsburgh Steelers': 'Pittsburgh Steelers',
    'San Diego Chargers': 'Los Angeles Chargers',  # Will be replaced
    'San Francisco 49ers': 'San Francisco 49ers',
    'Seattle Seahawks': 'Seattle Seahawks',
    'St. Louis Rams': 'Los Angeles Rams',  # Will be replaced
    'Tampa Bay Buccaneers': 'Tampa Bay Buccaneers',
    'Tennessee Titans': 'Tennessee Titans',
    'Washington Commanders': 'Washington Commanders',  # Updated
    'Washington Football Team': 'Washington Commanders',  # Will be replaced
    'Washington Redskins': 'Washington Commanders'  # Will be replaced
}

In [108]:
df['opp'] = df['opp'].map(team_mapping)

In [111]:
df

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,2,23,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,2,164,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,6,137,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,7,206,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,2023,WAS,13,Sun,December 3,L,no,home,Miami Dolphins,15,...,3,127,7,13,2,2,32:16,2023_13_Miami Dolphins_Washington Commanders,Washington Commanders,Miami Dolphins
12346,2023,WAS,15,Sun,December 17,L,no,away,Los Angeles Rams,20,...,3,134,9,16,0,0,35:53,2023_15_Los Angeles Rams_Washington Commanders,Washington Commanders,Los Angeles Rams
12347,2023,WAS,16,Sun,December 24,L,no,away,New York Jets,28,...,6,292,4,16,1,2,36:16,2023_16_New York Jets_Washington Commanders,Washington Commanders,New York Jets
12348,2023,WAS,17,Sun,December 31,L,no,home,San Francisco 49ers,10,...,2,76,4,9,0,1,38:13,2023_17_San Francisco 49ers_Washington Commanders,Washington Commanders,San Francisco 49ers


In [112]:
#Create a unique identifier for each game
# This ensures each game is represented only once, even if it's recorded from both teams' perspectives
df['game_id'] = df.apply(lambda row: f"{row['season']}_{row['week']}_{'_'.join(sorted([row['team_full_name'], row['opp']]))}", axis=1)

In [113]:
df

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,2,23,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,2,164,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,6,137,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,7,206,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,2023,WAS,13,Sun,December 3,L,no,home,Miami Dolphins,15,...,3,127,7,13,2,2,32:16,2023_13_Miami Dolphins_Washington Commanders,Washington Commanders,Miami Dolphins
12346,2023,WAS,15,Sun,December 17,L,no,away,Los Angeles Rams,20,...,3,134,9,16,0,0,35:53,2023_15_Los Angeles Rams_Washington Commanders,Washington Commanders,Los Angeles Rams
12347,2023,WAS,16,Sun,December 24,L,no,away,New York Jets,28,...,6,292,4,16,1,2,36:16,2023_16_New York Jets_Washington Commanders,Washington Commanders,New York Jets
12348,2023,WAS,17,Sun,December 31,L,no,home,San Francisco 49ers,10,...,2,76,4,9,0,1,38:13,2023_17_San Francisco 49ers_Washington Commanders,Washington Commanders,San Francisco 49ers


In [114]:
# Step 2: Drop duplicate entries based on the game_id
# Keep only one entry for each game (e.g., keeping the first occurrence)
df_cleaned = df.drop_duplicates(subset='game_id')

# Step 3: Drop the 'game_id' column if it's no longer needed
#df_cleaned = df_cleaned.drop(columns=['game_id'])

In [115]:
df_cleaned

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,2,23,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,2,164,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,6,137,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,7,206,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12277,2023,SEA,14,Sun,December 10,L,no,away,San Francisco 49ers,16,...,5,246,6,11,0,0,29:15,2023_14_San Francisco 49ers_Seattle Seahawks,Seattle Seahawks,San Francisco 49ers
12279,2023,SEA,16,Sun,December 24,W,no,away,Tennessee Titans,20,...,3,117,6,12,1,1,33:51,2023_16_Seattle Seahawks_Tennessee Titans,Seattle Seahawks,Tennessee Titans
12291,2023,SFO,11,Sun,November 19,W,no,home,Tampa Bay Buccaneers,27,...,3,144,7,14,1,3,29:09,2023_11_San Francisco 49ers_Tampa Bay Buccaneers,San Francisco 49ers,Tampa Bay Buccaneers
12297,2023,SFO,17,Sun,December 31,W,no,away,Washington Commanders,27,...,3,156,3,9,0,1,21:47,2023_17_San Francisco 49ers_Washington Commanders,San Francisco 49ers,Washington Commanders


In [116]:
df.shape

(12350, 66)

In [117]:
df_cleaned.shape

(6175, 66)

In [118]:
12350/2

6175.0

In [119]:
df_cleaned.sort_values(by = ['season' , 'week']).head(18)

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
16,2000,ATL,1,Sun,September 3,W,no,home,San Francisco 49ers,36,...,5,98,2,9,0,1,28:21,2000_1_Atlanta Falcons_San Francisco 49ers,Atlanta Falcons,San Francisco 49ers
32,2000,RAV,1,Sun,September 3,W,no,away,Pittsburgh Steelers,16,...,6,184,4,14,0,2,24:53,2000_1_Baltimore Ravens_Pittsburgh Steelers,Baltimore Ravens,Pittsburgh Steelers
48,2000,BUF,1,Sun,September 3,W,no,home,Tennessee Titans,16,...,9,308,2,14,0,1,26:49,2000_1_Buffalo Bills_Tennessee Titans,Buffalo Bills,Tennessee Titans
64,2000,CAR,1,Sun,September 3,L,no,away,Washington Commanders,17,...,4,64,5,12,0,1,33:14,2000_1_Carolina Panthers_Washington Commanders,Carolina Panthers,Washington Redskins
80,2000,CHI,1,Sun,September 3,L,no,away,Minnesota Vikings,27,...,2,156,6,13,0,0,27:26,2000_1_Chicago Bears_Minnesota Vikings,Chicago Bears,Minnesota Vikings
112,2000,CLE,1,Sun,September 3,L,no,home,Jacksonville Jaguars,7,...,3,35,10,16,0,0,37:03,2000_1_Cleveland Browns_Jacksonville Jaguars,Cleveland Browns,Jacksonville Jaguars
128,2000,DAL,1,Sun,September 3,L,no,home,Philadelphia Eagles,14,...,2,176,7,12,0,0,39:30,2000_1_Dallas Cowboys_Philadelphia Eagles,Dallas Cowboys,Philadelphia Eagles
144,2000,DEN,1,Mon,September 4,L,no,away,Los Angeles Rams,36,...,1,34,6,7,0,0,27:47,2000_1_Denver Broncos_Los Angeles Rams,Denver Broncos,St. Louis Rams
160,2000,DET,1,Sun,September 3,W,no,away,New Orleans Saints,14,...,7,240,6,17,0,1,29:11,2000_1_Detroit Lions_New Orleans Saints,Detroit Lions,New Orleans Saints


In [74]:
pd.options.display.max_rows = 20

In [122]:
df_cleaned.sort_values(by = ['season' , 'week'])

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
16,2000,ATL,1,Sun,September 3,W,no,home,San Francisco 49ers,36,...,5,98,2,9,0,1,28:21,2000_1_Atlanta Falcons_San Francisco 49ers,Atlanta Falcons,San Francisco 49ers
32,2000,RAV,1,Sun,September 3,W,no,away,Pittsburgh Steelers,16,...,6,184,4,14,0,2,24:53,2000_1_Baltimore Ravens_Pittsburgh Steelers,Baltimore Ravens,Pittsburgh Steelers
48,2000,BUF,1,Sun,September 3,W,no,home,Tennessee Titans,16,...,9,308,2,14,0,1,26:49,2000_1_Buffalo Bills_Tennessee Titans,Buffalo Bills,Tennessee Titans
64,2000,CAR,1,Sun,September 3,L,no,away,Washington Commanders,17,...,4,64,5,12,0,1,33:14,2000_1_Carolina Panthers_Washington Commanders,Carolina Panthers,Washington Redskins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12060,2023,JAX,18,Sun,January 7,L,no,away,Tennessee Titans,20,...,3,144,6,11,0,1,27:43,2023_18_Jacksonville Jaguars_Tennessee Titans,Jacksonville Jaguars,Tennessee Titans
12077,2023,KAN,18,Sun,January 7,W,no,away,Los Angeles Chargers,13,...,5,218,7,19,2,3,32:55,2023_18_Kansas City Chiefs_Los Angeles Chargers,Kansas City Chiefs,Los Angeles Chargers
12111,2023,RAM,18,Sun,January 7,W,no,away,San Francisco 49ers,21,...,3,144,5,10,1,1,29:00,2023_18_Los Angeles Rams_San Francisco 49ers,Los Angeles Rams,San Francisco 49ers
12179,2023,NWE,18,Sun,January 7,L,no,home,New York Jets,3,...,7,322,3,15,0,1,33:42,2023_18_New England Patriots_New York Jets,New England Patriots,New York Jets


In [123]:
# Step 4: Save the cleaned dataset to a new CSV file
df_cleaned.to_csv('cleaned_nfl_dataset.csv', index=False)
